# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient()

In [3]:
# assign the uk_food database to a variable name
uk_food = mongo['uk_food']

In [4]:
# review the collections in our database
uk_food.collection_names()

/var/folders/06/47q5ms9540g4mnz8nnnyl32m0000gn/T/ipykernel_16871/1426522449.py:2: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  uk_food.collection_names()


['establishments']

In [5]:
# assign the collection to a variable
establishments = uk_food['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

In [6]:
# Treating the RatingValue column within the data
values_to_replace = list(establishments.distinct('RatingValue'))

values_to_replace

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 'Awaiting Inspection',
 'AwaitingInspection',
 'AwaitingPublication',
 'Exempt',
 'Pass']

In [7]:
# Replacing the numerical string values to integers
establishments.update_many({'RatingValue':
                            {'$in': values_to_replace[0:6]}},
                            [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}])

In [8]:
# Replaceing the string values
establishments.update_many({'RatingValue':
                            {'$in': values_to_replace[6:]}},
                            [{'$set': {'RatingValue': None}}])

In [9]:
# Confirming the transformation process
establishments.distinct('RatingValue')

[None, 0, 1, 2, 3, 4, 5]

### 1. Which establishments have a hygiene score equal to 20?

In [10]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
range_count = establishments.count_documents(query)
range_count

41

In [11]:
# Display the first document in the results using pprint
results = establishments.find(query)

pprint(results[0])

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64e299f70ef6dfdcc495c4b9'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate

In [12]:
# Convert the result to a Pandas DataFrame
df = pd.json_normalize(results)
# # Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")
# # Display the first 10 rows of the DataFrame
df.head(10)

Number of rows in the DataFrame: 41


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,64e299f70ef6dfdcc495c4b9,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,0.276940,50.769705,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,64e299f70ef6dfdcc495c83b,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,1.278721,51.857536,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,64e299f70ef6dfdcc495cb68,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,1.159270,51.789429,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,64e299f70ef6dfdcc495cd31,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,0.102020,50.770885,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,64e299f70ef6dfdcc495cd3c,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,0.101446,50.770724,None,0001-01-01T00:00:00,0,None,0,0,0,0
5,64e299f80ef6dfdcc495d6e1,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,0.736349,51.541448,None,0001-01-01T00:00:00,0,None,0,0,0,0
6,64e299f80ef6dfdcc495d902,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,0.712167,51.535007,None,0001-01-01T00:00:00,0,None,0,0,0,0
7,64e299f80ef6dfdcc495ee2a,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,0.602364,51.591515,None,0001-01-01T00:00:00,0,None,0,0,0,0
8,64e299f80ef6dfdcc495f26d,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,0.146450,51.185947,None,0001-01-01T00:00:00,0,None,0,0,0,0
9,64e299f80ef6dfdcc495fa82,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,0.462307,51.570050,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [13]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'$and': [{'LocalAuthorityName': {'$regex': 'London'}}, {'RatingValue': {'$gte':4}}]}

# Use count_documents to display the number of documents in the result
range_count = establishments.count_documents(query)
# * Print the results of range_count
print(f"""
There are {range_count} documents that matches these specifications.
""")
# Display the first document in the results using pprint
results = establishments.find(query)

pprint(results[0])


There are 33 documents that matches these specifications.

{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64e299f80ef6dfdcc495ded2'),
 'geocode': {'latitude': 51.369321, 'longitude': 0.508551},
 'links': [{'href': 'https://api.ratings.foo

In [14]:
# Convert the result to a Pandas DataFrame
df_2 = pd.json_normalize(results)
# # Display the number of rows in the DataFrame
print(f"""
There are {df_2.shape[0]} rows in the DataFrame
""")
# Display the first 10 rows of the DataFrame
df_2.head(10)



There are 33 rows in the DataFrame



,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,64e299f80ef6dfdcc495ded2,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,...,0.508551,51.369321,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,64e299f80ef6dfdcc495e1f3,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,0.249256,51.123623,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,64e299f80ef6dfdcc495ed48,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,0.561954,51.543831,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,64e299f80ef6dfdcc495fb48,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,0.346325,51.464078,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,64e299f80ef6dfdcc495fb49,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,0.346337,51.464066,None,0001-01-01T00:00:00,0,None,0,0,0,0
5,64e299f80ef6dfdcc496024e,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,0.287555,51.504071,None,0001-01-01T00:00:00,0,None,0,0,0,0
6,64e299f80ef6dfdcc496235d,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,0.069286,51.501121,None,0001-01-01T00:00:00,0,None,0,0,0,0
7,64e299f80ef6dfdcc4962726,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,0.048006,51.503733,None,0001-01-01T00:00:00,0,None,0,0,0,0
8,64e299f80ef6dfdcc496273d,293772,0,PI/000002585,The Nuance Group (UK) Limited,Retailers - other,4613,Duty Free Shop Passenger Terminal,London City AirportRoyal DocksLondon,,...,0.047966,51.504112,None,0001-01-01T00:00:00,0,None,0,0,0,0
9,64e299f80ef6dfdcc4962743,294606,0,PI/000016185,WH Smith,Retailers - other,4613,London City Airport,Silvertown London,,...,0.047966,51.504112,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [15]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000
rating_value = 5

query = query = {
    'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
    'RatingValue': rating_value
}
sort = [('scores.Hygiene', 1)]

# Print the results
range_count = establishments.count_documents(query)
print(f"""
There are {range_count} documents
""")
results = establishments.find(query).sort(sort).limit(5)

pprint(results[0])


There are 87 documents

{'AddressLine1': '144 - 146 Plumstead High Street',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'AddressLine4': 'Greenwich',
 'BusinessName': 'Iceland',
 'BusinessType': 'Retailers - supermarkets/hypermarkets',
 'BusinessTypeID': 7840,
 'ChangesByServerID': 0,
 'Distance': 4646.946071297699,
 'FHRSID': 695223,
 'LocalAuthorityBusinessID': 'PI/000178842',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE18 1JQ',
 'RatingDate': '2019-11-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64e299f80ef6dfdcc4961cec'),
 'geocode': {'latitude': 51.4871482849121, 'longitude': 0.0924199968576431},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/695223',
            'rel': 'se

In [16]:
# Convert result to Pandas DataFrame
df_3 = pd.json_normalize(results)
# # Display the number of rows in the DataFrame
print(f"""
There are {df_3.shape[0]} rows in the DataFrame
""")
# Display the first 10 rows of the DataFrame
df_3.head(10)


There are 5 rows in the DataFrame



,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,64e299f80ef6dfdcc4961cf9,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,...,0.092537,51.487534,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,64e299f80ef6dfdcc4961d33,694478,0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,...,0.091216,51.486730,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,64e299f80ef6dfdcc4961d2b,695241,0,PI/000179088,Plumstead Manor Nursery,Caring Premises,5,Plumstead Manor School Old Mill Road,,Plumstead,...,0.085994,51.481518,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,64e299f80ef6dfdcc4961cec,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,...,0.092420,51.487148,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,64e299f80ef6dfdcc4961d19,694609,0,PI/000116619,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,...,0.092080,51.487344,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [17]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {'scores.Hygiene':0}}

group_query = {'$group': {'_id': '$LocalAuthorityName', 'count':{'$sum': 1}}}

sort_query = {'$sort': {'count': -1}}

pipeline= [match_query, group_query, sort_query]

results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print(f"""
Number of documents: {len(results)}
""")
# Print the first 10 results
results[:10]


Number of documents: 55



[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]

In [18]:
# Convert the result to a Pandas DataFrame
aggregated_df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {aggregated_df.shape[0]}")

# Display the first 10 rows of the DataFrame
aggregated_df.head(10)

Number of rows in the DataFrame: 55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
